In [1]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

import re
import xml.etree.ElementTree as ET
import pandas as pd

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *
pkg

`isr.umich.edu-psid-inequality-1.1.2` Last Update: 2023-06-08T23:18:30

__



## Updating Data

To update the data, use the data extraction tool at the (PSID website. )[https://simba.isr.umich.edu/Cart/co.aspx], adding any new variables to your data cart. The (variable entry text box)[https://simba.isr.umich.edu/DC/l.aspx] is a handy way to copy the existing varibles into a new cart.  When you checkout your data cart, ( click on the cart icon ) fetch your dataset with these options: 

* Codebook Type: XML
* Data output type: dBase Data File, create compressed file

Download the ZIP file ( you might have to recompress it if your browser unpacks it ) and move it into the ``data`` directory. Set the reference to the new ZIP file, then re-build the package.
## Documentation Links

* [PSID Homepage](https://psidonline.isr.umich.edu/default.aspx) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[psid_ineq](notebooks/extract.ipynb#df)**. PSID, Inequality extract
* **[_labels](notebooks/extract.ipynb#labels)**. Categorical value labels
* **[data_dictionary](notebooks/extract.ipynb#mdf)**. List of Variables

## References
<ul><li> <strong><a href="https://psidonline.isr.umich.edu/help/xyr/psid.xlsx">variables</a></strong>. Variable descriptions, in a cross-year format.</li><li> <strong>source</string>, <em>data/J321089.zip</em>. Downloaded source file</li><ul>

In [3]:
# create URLs to the files inside the data distribution ZIP file
surl   = pkg.reference('source').resolved_url
base_name = surl.resource_file.split('.')[0]
database_url = surl.set_target_file(base_name+'.dbf')

from simpledbf import Dbf5
dbf = Dbf5(database_url.get_target().fspath)
df = dbf.to_dataframe()

# Make everything as numerical as possible
for c in df.columns:
    df[c] = pd.to_numeric(df[c])

df = df.copy()

# Recommended construction of unique person id
df['pid'] = (df['ER30001']*1000) + df['ER30002']


PyTables is not installed. No support for HDF output.


In [4]:
# opening_columns. These should always be at the start of the schema.
init_cols = ['ER30000','ER30001','ER30002']

In [5]:
varis, mdf = extract_metadata(pkg)
labels = extract_labels(pkg)

In [21]:
from pylib import *

lookup_vars(varis, init_cols)


['ER30001', 'ER30002', 'ER30000']

In [23]:
# Re-order the data dictionary


,year,type_id,name,label,qtext,etext,type,category,text,head_wife,var_count,group,path
1,1968,0,ER30001,1968 INTERVIEW NUMBER,1968 Interview Number (1968 ID Number),This variable is the 1968 family ID number. T...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,418,/survey information:/interview information/id ...
2,1968,0,ER30002,PERSON NUMBER 68,Person Number,,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,420,/survey information:/interview information/per...
0,1968,0,ER30000,RELEASE NUMBER,Release Number,For Release 2 we have updated Relation to Hea...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,422,/survey information:/interview information/rel...
28,1999,2,ER33501,1999 INTERVIEW NUMBER,1999 Interview Number (1999 ID Number),The values for this variable represent the 199...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,418,/survey information:/interview information/id ...
64,2019,2,ER34701,2019 INTERVIEW NUMBER,2019 Interview Number (2019 ID Number),The values for this variable represent the 20...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,418,/survey information:/interview information/id ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2019,1,ER77448,TOTAL FAMILY INCOME-2018,Total 2018 Family Money Income,The income reported here was collected in 201...,FAMILY PUBLIC,INCOME,01>INCOME\n02>Family Money\n03>total family in...,,41.0,5562,/income/family money/total family income:
46,2019,1,ER77507,IMP VALUE HOME EQUITY 2019,Constructed value of home equity.,Constructed value of home equity. This imput...,FAMILY PUBLIC,WEALTH,01>WEALTH\n02>Home Equity\n03>value:,,14.0,9047,/wealth/home equity/value:
26,1999,1,S420,IMP VALUE HOME EQUITY 99,,Constructed value of home equity. This impute...,FAMILY PUBLIC,WEALTH,01>WEALTH\n02>Home Equity\n03>value:,,14.0,9047,/wealth/home equity/value:
47,2019,1,ER77511,IMP WEALTH W/ EQUITY (WEALTH2) 2019,"Constructed Wealth Variable, Including Equity.","Constructed wealth variable, including equity...",FAMILY PUBLIC,WEALTH,01>WEALTH\n02>Total Family Wealth\n03>includin...,,14.0,9235,/wealth/total family wealth/including home equ...


In [8]:
df = t